# Imports

In [7]:
import os
import warnings
warnings.filterwarnings("ignore")

from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import scipy as sp
import scipy.signal
import scipy.stats
from scipy.stats.mstats import gmean

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from sklearn.model_selection import GridSearchCV, LeaveOneOut, StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from scipy.stats import mode
from sklearn.model_selection import StratifiedKFold, GridSearchCV, LeaveOneOut

from utils import Dataset, variance_thresholding, standardize, mcc, calculate_metrics, calculate_metrics_statistics

In [9]:
# parameters for Welch's method for estimating power spectrum

NPERSEG = 24                    # length of segment
NOVERLAP = int(0.75 * NPERSEG)  # overlap of segments
NFFT = NPERSEG                  # length of FFT
WINDOW = "hann"                 # window function type

# parameters for saving data
PROCESSED_DATA_DIR = "processed_data24h"
DEPRESJON_PREFIX = "manual_depresjon24h"
PSYKOSE_PREFIX = "manual_psykose24h"

# Manual feature extraction

## Helper functions

In [10]:
def basic_data_cleaning(data: List[pd.DataFrame]) -> List[pd.DataFrame]:
    """
    Assumes DataFrames with "timestamp", "date" and "activity" columns.
    
    Performs cleaning operations:
    - assure format YYYY-MM-DD HH:MM:SS for "timestamp"
    - drop redundant "date" column
    - assure float32 format for "activity"
    
    :param data: list of DataFrames
    :returns: list of cleaned DataFrames
    """
    data = [df.copy() for df in data]  # create copy to avoid side effects
    
    for df in data:
        df["timestamp"] = pd.to_datetime(df["timestamp"],
                                         format="%Y-%m-%d %H:%M:%S")
        df.drop("date", axis=1, inplace=True)
        df["activity"] = df["activity"].astype(np.float32)
    
    return data


def get_day_part(df: pd.DataFrame, part: str) -> pd.DataFrame:
    """
    For given DataFrame with "timestamp" column returns only those rows that
    correspond to the chosen part of day.
    
    Parts are "day" and "night", defined as:
    - "day": [8:00, 21:00)
    - "night": [21:00, 8:00)
    
    :param df: DataFrame to select rows from
    :param part: part of day, either "day" or "night"
    :returns: DataFrame, subset of rows of df
    """
    if part == "day":
        df = df.loc[(df["timestamp"].dt.hour >= 8) &
                    (df["timestamp"].dt.hour < 21)]
    elif part == "night":
        df = df.loc[(df["timestamp"].dt.hour >= 21) |
                    (df["timestamp"].dt.hour < 8)]
    else:
        raise ValueError(f'Part should be "day" or "night", got "{part}"')
        
    return df


def fill_missing_activity(df: pd.DataFrame) -> pd.DataFrame:
    """
    Makes sure that "timestamp" column has minute resolution with no missing
    values from start to end and replaces all NaNs in "activity" column with
    mean average value.
    
    :param data: DataFrame with "timestamp" and "activity" columns
    :returns: cleaned DataFrame
    """
    df = df.copy()  # create copy to avoid side effects
    
    # resample to the basic frequency, i.e. minute; this will create NaNs for
    # any rows that may be missing
    df = df.resample("min", on="timestamp").mean()
    
    # recreate index and "timestamp" column
    df = df.reset_index()
    
    # fill any NaNs with mean activity value
    df["activity"] = df["activity"].fillna(df["activity"].mean())

    return df


def resample(df: pd.DataFrame, freq: str = "H") -> pd.DataFrame:
    """
    Resamples time series DataFrame with given frequency, aggregating each
    segment with a mean.

    :param df: DataFrame with "timestamp" and "activity" columns
    :param freq: resampling frequency passed to Pandas resample() function
    :returns: DataFrame with "timestamp" and "activity" columns
    """
    df = df.copy()  # create copy to avoid side effects
    
    # group with given frequency
    df = df.resample(freq, on="timestamp").mean()

    # recreate "timestamp" column
    df = df.reset_index()

    return df


def proportion_of_zeros(x: np.ndarray) -> float:
    """
    Calculates proportion of zeros in given array, i.e. number of zeros divided
    by length of array.
    
    :param x: 1D Numpy array
    :returns: proportion of zeros
    """
    # we may be dealing with floating numbers, we can't use direct comparison
    zeros_count = np.sum(np.isclose(x, 0))
    return zeros_count / len(x)


def power_spectral_density(df: pd.DataFrame) -> np.ndarray:
    """
    Calculates power spectral density (PSD) from "activity" column of a
    DataFrame.
    
    :param df: DataFrame with "activity" column
    :returns: 1D Numpy array with power spectral density
    """
    psd = scipy.signal.welch(
        x=df["activity"].values,
        fs=(1/24),
        nperseg=11,
        noverlap=10,
        nfft=NFFT,
        window=WINDOW,
        scaling="density"
    )[1]
    return psd


def spectral_flatness(df: pd.DataFrame) -> float:
    """
    Calculates spectral flatness of a signal, i.e. a geometric mean of the
    power spectrum divided by the arithmetic mean of the power spectrum.
    
    If some frequency bins in the power spectrum are close to zero, they are
    removed prior to calculation of spectral flatness to avoid calculation of
    log(0).
    
    :param df: DataFrame with "activity" column
    :returns: spectral flatness value
    """
    power_spectrum = scipy.signal.welch(
        df["activity"].values,
        fs=(1/24),
        nperseg=11,
        noverlap=10,
        nfft=NFFT,
        window=WINDOW,
        scaling="spectrum"
    )[1]
    
    non_zeros_mask = ~np.isclose(power_spectrum, 0)
    power_spectrum = power_spectrum[non_zeros_mask]
    
    return scipy.stats.gmean(power_spectrum) / power_spectrum.mean()

## Feature extraction

In [11]:
def extract_time_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts features from activity signal in time domain.
    
    :param df_resampled: DataFrame with "activity" column
    :returns: DataFrame with a single row representing features
    """
    X = df["activity"].values
    
    features = {
        "minimum": np.min(X),
        "maximum": np.max(X),
        "mean": np.mean(X),
        "median": np.median(X),
        "variance": np.var(X, ddof=1),  # apply Bessel's correction
        "kurtosis": sp.stats.kurtosis(X),
        "skewness": sp.stats.skew(X),
        "coeff_of_var": sp.stats.variation(X),
        "iqr": sp.stats.iqr(X),
        "trimmed_mean": sp.stats.trim_mean(X, proportiontocut=0.1),
        "entropy": sp.stats.entropy(X, base=2),
        "proportion_of_zeros": proportion_of_zeros(X)
    }
    
    return pd.DataFrame([features])

In [12]:
def extract_frequency_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts features from activity signal in frequency domain, i.e. calculated
    from its Power Spectral Density (PSD).
    
    :param df: DataFrame with "activity" column
    :returns: DataFrame with a single row representing features
    """
    X = power_spectral_density(df)
    
    features = {
        "minimum": np.min(X),
        "maximum": np.max(X),
        "mean": np.mean(X),
        "median": np.median(X),
        "variance": np.var(X),
        "kurtosis": sp.stats.kurtosis(X),
        "skewness": sp.stats.skew(X),
        "coeff_of_var": sp.stats.variation(X),
        "iqr": sp.stats.iqr(X),
        "trimmed_mean": sp.stats.trim_mean(X, proportiontocut=0.1),
        "entropy": sp.stats.entropy(X, base=2),
        "spectral_flatness": spectral_flatness(df)
    }
    
    return pd.DataFrame([features])

In [13]:
def extract_features_for_dataframes(dfs: List[pd.DataFrame], freq: str = "H") \
        -> Dict[str, pd.DataFrame]:
    """
    Calculates time and frequency features for given DataFrames. Uses given
    frequency for resampling.
    
    Calculates features separately for:
    - full 24hs
    - days: [8:00, 21:00)
    - nights: [21:00, 8:00)
    
    :param dfs: list of DataFrames to extract features from; each one has to
    have "timestamp" and "activity" columns
    :param freq: resampling frequency
    :returns: dictionary with keys "full_24h", "day" and "night", corresponding
    to features from given parts of day
    """
    full_dfs = basic_data_cleaning(dfs)
    full_dfs = [fill_missing_activity(df) for df in full_dfs]
    full_dfs = [resample(df, freq=freq) for df in full_dfs]
    
    night_dfs = [get_day_part(df, part="night") for df in full_dfs]
    day_dfs = [get_day_part(df, part="day") for df in full_dfs]

    datasets = {}
    
    for part, list_of_dfs in [("full_24h", full_dfs), ("night", night_dfs),
                              ("day", day_dfs)]:
        features = []
        for df in list_of_dfs:
            time_features = extract_time_features(df)
            freq_features = extract_frequency_features(df)

            merged_features = pd.merge(
                time_features,
                freq_features,
                left_index=True,
                right_index=True,
                suffixes=["_time", "_freq"]
            )
            features.append(merged_features)

        datasets[part] = pd.concat(features)
        datasets[part].reset_index(drop=True, inplace=True)
    
    return datasets



## Depresjon

In [14]:
dataset = Dataset(dirpath=os.path.join("data_24h", "depresjon"))
condition = dataset.condition
control = dataset.control

In [15]:
condition[0]

,timestamp,date,activity
0,2003-05-07 12:00:00,2003-05-07,0
1,2003-05-07 12:01:00,2003-05-07,143
2,2003-05-07 12:02:00,2003-05-07,0
3,2003-05-07 12:03:00,2003-05-07,20
4,2003-05-07 12:04:00,2003-05-07,166
...,...,...,...
1435,2003-05-08 11:55:00,2003-05-08,259
1436,2003-05-08 11:56:00,2003-05-08,190
1437,2003-05-08 11:57:00,2003-05-08,306
1438,2003-05-08 11:58:00,2003-05-08,91


In [16]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["full_24h"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]
    
    entire_df = pd.concat([condition_df, control_df], ignore_index=True)
    
    # Przypisujemy wynik do słownika datasets
    datasets[part] = entire_df

In [17]:
for part, df in datasets.items():
    filename = f"{DEPRESJON_PREFIX}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    df.to_csv(filepath, index=False)

In [18]:
condition_df

,minimum_time,maximum_time,mean_time,median_time,variance_time,kurtosis_time,skewness_time,coeff_of_var_time,iqr_time,trimmed_mean_time,...,mean_freq,median_freq,variance_freq,kurtosis_freq,skewness_freq,coeff_of_var_freq,iqr_freq,trimmed_mean_freq,entropy_freq,spectral_flatness
0,0.966667,395.649994,131.482635,105.699997,16655.847656,-0.920654,0.638463,0.960889,212.062506,120.149170,...,145556.015625,94128.164062,1.245808e+10,0.335413,1.309549,0.766824,68011.867188,131411.703125,3.339275,0.781018
1,2.083333,432.483337,153.253464,97.983337,20828.339844,-1.155962,0.531038,0.921882,247.795844,142.218353,...,225564.921875,66185.421875,7.144961e+10,-0.185189,1.162469,1.185027,291117.929688,191007.359375,2.800678,0.442222
2,0.583333,339.666656,124.177773,99.625000,12966.193359,-1.028185,0.526891,0.897678,184.895827,115.246666,...,337368.875000,233032.718750,9.313090e+10,-0.446068,0.971081,0.904569,379323.562500,306780.781250,3.145580,0.615705
3,0.916667,447.583344,128.460403,101.724998,14834.280273,0.110828,0.837585,0.928159,193.966663,114.235825,...,293284.031250,243482.046875,2.841528e+10,-1.273162,0.381854,0.574761,266578.562500,286987.468750,3.457804,0.826515
4,2.666667,368.250000,102.386108,59.950001,11585.225586,0.219186,1.074547,1.029128,136.641668,88.289169,...,75315.054688,50414.445312,2.197785e+09,0.339903,1.245119,0.622459,29237.988281,69791.164062,3.455590,0.847506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,1.333333,635.616638,223.734741,204.725006,44670.164062,-1.252771,0.421382,0.924770,386.612508,208.136673,...,502913.000000,319598.937500,1.322892e+11,-1.131078,0.579552,0.723218,545265.203125,481347.187500,3.324815,0.737595
368,0.000000,896.299988,123.432640,0.000000,66508.078125,3.024749,2.085160,2.045340,50.679169,64.109169,...,643649.187500,177262.453125,6.870181e+11,-0.371501,1.079565,1.287760,831232.622070,537330.375000,2.543363,0.199196
369,0.000000,12.933333,1.077083,0.000000,7.640334,12.440123,3.530067,2.512264,0.487500,0.460000,...,387.263550,422.427948,2.624917e+04,-0.764357,-0.678815,0.418361,234.294464,400.103149,3.546332,0.857989
370,0.000000,7.000000,0.408333,0.000000,2.066377,17.069595,4.271915,3.446262,0.012500,0.080000,...,13.137053,11.149598,1.006334e+02,-0.397369,0.965084,0.763613,8.682478,12.300607,3.309889,0.739117


In [19]:
y = np.concatenate((np.ones(len(condition)), np.zeros(len(control))))
y = pd.Series(y, dtype=int)

filepath = os.path.join(PROCESSED_DATA_DIR, f"depresjon_y.csv")
y.to_csv(filepath, header=False, index=False)

In [20]:
# to get ensemble:
import os

def sort_key(filename):
    parts = filename.split("_")
    # Wydobycie numerów jako liczb całkowitych
    condition_number = int(parts[1])  # Druga część np. "1" z "condition_1"
    segment_number = int(parts[3].split(".")[0])  # Trzecia część np. "1" z "segmen1.csv"
    return (condition_number, segment_number)


In [21]:
# Removing NaNs
folder_control = './data_24h/depresjon/control'
folder_condition = './data_24h/depresjon/condition'
z_control = []
z_condition = []
for filename in sorted(os.listdir(folder_control), key=sort_key):
    z_control.append(filename.split("_")[0] + filename.split("_")[1])
    
for filename in sorted(os.listdir(folder_condition), key=sort_key):
    z_condition.append(filename.split("_")[0] + filename.split("_")[1])
    
z_control = pd.Series(z_control)
z_condition = pd.Series(z_condition)

z = pd.concat([z_condition, z_control])
z.index = datasets["full_24h"].index

mask = datasets["full_24h"].notna().all(axis=1)
datasets["full_24h"] = datasets["full_24h"][mask]
y = y[mask]
z_depresjon = z[mask]

## Psykose

In [22]:
dataset = Dataset(dirpath=os.path.join("data_24h", "psykose"))
condition = dataset.condition
control = dataset.control

In [23]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["full_24h"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]

    entire_df = pd.concat([condition_df, control_df], ignore_index=True)

    # Przypisujemy wynik do słownika datasets
    datasets[part] = entire_df

In [24]:
for part, df in datasets.items():
    filename = f"{PSYKOSE_PREFIX}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    df.to_csv(filepath, index=False)

In [25]:
y = np.concatenate((np.ones(len(condition)), np.zeros(len(control))))
y = pd.Series(y, dtype=int)

filepath = os.path.join(PROCESSED_DATA_DIR, f"psykose_y.csv")
y.to_csv(filepath, header=False, index=False)
#datasets["full_24h"].dropna(inplace=True)

In [26]:
# Removing NaNs
folder_control = './data_24h/psykose/control'
folder_condition = './data_24h/psykose/condition'
z_control = []
z_condition = []
for filename in sorted(os.listdir(folder_control), key=sort_key):
    z_control.append(filename.split("_")[0] + filename.split("_")[1])
    
for filename in sorted(os.listdir(folder_condition), key=sort_key):
    z_condition.append(filename.split("_")[0] + filename.split("_")[1])
    
z_control = pd.Series(z_control)
z_condition = pd.Series(z_condition)

z = pd.concat([z_condition, z_control])
z.index = datasets["full_24h"].index
mask = datasets["full_24h"].notna().all(axis=1)
datasets["full_24h"] = datasets["full_24h"][mask]
y = y[mask]
z_psykose = z[mask]

# Classification

## Classifiers, parameters, constants

In [37]:
classifiers = {
    "LR": LogisticRegression(
        penalty="elasticnet",
        random_state=0,
        solver="saga",
        max_iter=500
    ),
    "SVM": SVC(
        kernel="rbf",
        cache_size=512
    ),
    "RF": RandomForestClassifier(
        n_estimators=200,
        criterion="entropy"
    ),
    "LGBM": LGBMClassifier(
        n_estimators=200,
        verbosity=-1,
        random_state=0
    ),
    "XGB": XGBClassifier(
        n_estimators=200,
        random_state=0 
    )
}


param_grids = {
    "LR": {
        "C": [0.001, 0.01, 0.1, 0.5, 1, 2, 5, 10, 25, 50, 100, 500, 1000],
        "class_weight": ["balanced"],
        "l1_ratio": [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5,
                     0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]
    },
    "SVM": {
        "class_weight": [None, "balanced"]
    },
    "RF": {
        "class_weight": [None, "balanced", "balanced_subsample"]
    },
    "LGBM": {
        "class_weight": [None, "balanced"]
    },
    "XGB": {
        
    }
}

## Classification

# Classification - Depresjon

In [28]:
dataset = DEPRESJON_PREFIX  # DEPRESJON_PREFIX or PSYKOSE_PREFIX
y_filename = "depresjon_y.csv" if dataset == DEPRESJON_PREFIX else "psykose_y.csv"

In [29]:
# Assuming PROCESSED_DATA_DIR and y_filename are already defined
datasets = {}

# Load datasets
for part in ["full_24h"]:
    filename = f"{dataset}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0).values

# Load y values
y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, y_filename), header=None, dtype=int)
y = y.values.ravel()
# Usuwanie wierszy, które zawierają NaN w danych
for part in datasets:
    # Indeksy wierszy, które zawierają NaN w dowolnej kolumnie
    nan_indices = np.isnan(datasets[part]).any(axis=1)
    
    # Usuwamy te wiersze z datasets i y
    datasets[part] = datasets[part][~nan_indices]
    y = y[~nan_indices]

# Sprawdzamy kształt danych po usunięciu NaN
#print(datasets['full_24h'].shape)
#print(y.shape)
#print(z_depresjon.shape)

In [38]:
print("Wyniki dla Depresjon:")
for part in ["full_24h"]:
    print(f"PART: {part}")
    
    X = datasets[part]
    for clf_type in ["LR", "SVM", "RF", "LGBM", "XGB"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        
        test_scores = []
        group_results = {}  
        
        for train_idx, test_idx in folds.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx] 
            y_train, y_test = y[train_idx], y[test_idx]
            
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="f1_weighted",
                n_jobs=-1,
                refit=True,
                #cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)

            clf = grid_search.best_estimator_
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            for i in range(len(test_idx)):
                
                condition_id = z_depresjon.iloc[test_idx[i]] 
                class_label = y[test_idx[i]] 
                unique_id = (condition_id, class_label)  

                if unique_id not in group_results:
                    group_results[unique_id] = []  
                group_results[unique_id].append(y_pred[i])  
            final_predictions = {}
            for unique_id, preds in group_results.items():
                condition_id, class_label = unique_id
                majority_vote = mode(preds).mode
                final_predictions[unique_id] = majority_vote 
            y_true_grouped = []
            y_pred_grouped =  []
            for k,v in final_predictions.items():
                y_true_grouped.append(k[1])
                y_pred_grouped.append(v)
            metrics = calculate_metrics(y_true_grouped, y_pred_grouped)
            test_scores.append(metrics)
            
        final_scores = calculate_metrics_statistics(test_scores)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()

Wyniki dla Depresjon:
PART: full_24h
  LR
    accuracy: 0.5046 +- 0.0764
    balanced_accuracy: 0.4864 +- 0.0309
    f1: 0.3137 +- 0.1550
    precision: 0.4041 +- 0.0882
    recall: 0.3540 +- 0.2885
    specificity: 0.6188 +- 0.3202
    ROC_AUC: 0.4864 +- 0.0309
    MCC: -0.0368 +- 0.0879

  SVM
    accuracy: 0.6401 +- 0.0683
    balanced_accuracy: 0.6027 +- 0.0730
    f1: 0.4712 +- 0.1018
    precision: 0.6054 +- 0.1412
    recall: 0.3867 +- 0.0810
    specificity: 0.8187 +- 0.0696
    ROC_AUC: 0.6027 +- 0.0730
    MCC: 0.2309 +- 0.1623

  RF
    accuracy: 0.7398 +- 0.0257
    balanced_accuracy: 0.6933 +- 0.0294
    f1: 0.5732 +- 0.0494
    precision: 0.8929 +- 0.0887
    recall: 0.4240 +- 0.0445
    specificity: 0.9625 +- 0.0306
    ROC_AUC: 0.6933 +- 0.0294
    MCC: 0.4796 +- 0.0722

  LGBM
    accuracy: 0.7361 +- 0.0165
    balanced_accuracy: 0.6840 +- 0.0203
    f1: 0.5434 +- 0.0361
    precision: 0.9600 +- 0.0800
    recall: 0.3805 +- 0.0322
    specificity: 0.9875 +- 0.0250
    

# Classification - Psykose

In [39]:
dataset = PSYKOSE_PREFIX  # DEPRESJON_PREFIX or PSYKOSE_PREFIX
y_filename = "depresjon_y.csv" if dataset == DEPRESJON_PREFIX else "psykose_y.csv"

In [40]:
for part in ["full_24h"]:
    filename = f"{dataset}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0).values

# Load y values
y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, y_filename), header=None, dtype=int)
y = y.values.ravel()
# Usuwanie wierszy, które zawierają NaN w danych
for part in datasets:
    # Indeksy wierszy, które zawierają NaN w dowolnej kolumnie
    nan_indices = np.isnan(datasets[part]).any(axis=1)
    
    # Usuwamy te wiersze z datasets i y
    datasets[part] = datasets[part][~nan_indices]
    y = y[~nan_indices]

In [43]:
print("Wyniki dla Psykose:")
for part in ["full_24h"]:
    print(f"PART: {part}")
    
    X = datasets[part]
    for clf_type in ["LR", "SVM", "RF", "LGBM", "XGB"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        
        test_scores = []
        group_results = {}  
        
        for train_idx, test_idx in folds.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx] 
            y_train, y_test = y[train_idx], y[test_idx]
            
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="f1_weighted",
                n_jobs=-1,
                refit=True,
                #cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)

            clf = grid_search.best_estimator_
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            for i in range(len(test_idx)):
                
                condition_id = z_psykose.iloc[test_idx[i]] 
                class_label = y[test_idx[i]] 
                unique_id = (condition_id, class_label)  

                if unique_id not in group_results:
                    group_results[unique_id] = []  
                group_results[unique_id].append(y_pred[i])  
            final_predictions = {}
            for unique_id, preds in group_results.items():
                condition_id, class_label = unique_id
                majority_vote = mode(preds).mode # mode[0] stara wersja boiblioteki
                final_predictions[unique_id] = majority_vote 
            y_true_grouped = []
            y_pred_grouped =  []
            for k,v in final_predictions.items():
                y_true_grouped.append(k[1])
                y_pred_grouped.append(v)
            metrics = calculate_metrics(y_true_grouped, y_pred_grouped)
            test_scores.append(metrics)
            
        final_scores = calculate_metrics_statistics(test_scores)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()

Wyniki dla Psykose:
PART: full_24h
  LR
    accuracy: 0.4963 +- 0.0181
    balanced_accuracy: 0.5594 +- 0.0300
    f1: 0.5910 +- 0.0360
    precision: 0.4408 +- 0.0162
    recall: 0.9000 +- 0.0971
    specificity: 0.2188 +- 0.0395
    ROC_AUC: 0.5594 +- 0.0300
    MCC: 0.1713 +- 0.0947

  SVM
    accuracy: 0.6370 +- 0.0301
    balanced_accuracy: 0.5574 +- 0.0330
    f1: 0.2195 +- 0.0813
    precision: 0.9000 +- 0.2000
    recall: 0.1273 +- 0.0530
    specificity: 0.9875 +- 0.0250
    ROC_AUC: 0.5574 +- 0.0330
    MCC: 0.2408 +- 0.1084

  RF
    accuracy: 0.6852 +- 0.0331
    balanced_accuracy: 0.6264 +- 0.0391
    f1: 0.4404 +- 0.0876
    precision: 0.7811 +- 0.0727
    recall: 0.3091 +- 0.0727
    specificity: 0.9437 +- 0.0125
    ROC_AUC: 0.6264 +- 0.0391
    MCC: 0.3355 +- 0.0882

  LGBM
    accuracy: 0.6815 +- 0.0460
    balanced_accuracy: 0.6148 +- 0.0565
    f1: 0.3812 +- 0.1359
    precision: 0.8902 +- 0.1222
    recall: 0.2545 +- 0.1171
    specificity: 0.9750 +- 0.0234
    ROC